# Implementare una procedura di Data Splitting per calcolare dei p-value e CI onesti.

**Domanda**: Quanto guadagna un battitore in base alle sue statistiche?

Nel csv [Hitters.csv](/Homework_3/Hitters.csv) ci sono i dati di circa 300 battitori e 19 statistiche più lo stipendio.

In [53]:
import pandas as pd
import numpy as np

In [54]:
hitters = pd.read_csv(
    "Hitters.csv"
)
print(hitters.shape)
hitters.head()

(322, 20)


,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


## Preparazione dei dati
Abbiamo 3 variabili categoriche:

- League
- Division
- NewLeague

Verifichiamo quante classi ci sono in queste variabili e creiamo variabili dummies di conseguenza, escludendone una come riferimento.

In [55]:
print(f"League: {list(hitters['League'].unique())}")
print(f"Division: {list(hitters['Division'].unique())}")
print(f"NewLeague: {list(hitters['NewLeague'].unique())}")

League: ['A', 'N']
Division: ['E', 'W']
NewLeague: ['A', 'N']


Adesso che sappiamo le categorie possiamo aggiungere le dummies.

Essendo solo due classi a variabile categorica possiamo anche decidere di rinominare la colonna, ad esempio:
**League -> League_A** e mettere 1 se League è A e 0 altrimenti. Il ragionamento può essere iterato anche alle altre variabili.

In [56]:
hitters.rename(columns={'League': 'League_A', 'Division': 'Division_E','NewLeague': 'NewLeague_A'}, inplace=True)
hitters["League_A"] = np.where(hitters["League_A"] == "A", 1, 0)
hitters["Division_E"] = np.where(hitters["Division_E"] == "E", 1, 0)
hitters["NewLeague_A"] = np.where(hitters["NewLeague_A"] == "A", 1, 0)

In [57]:
hitters.dropna(inplace=True)
print(hitters.shape)
hitters.head()

(263, 20)


,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League_A,Division_E,PutOuts,Assists,Errors,Salary,NewLeague_A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,0,0,632,43,10,475.0,0
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,1,0,880,82,14,480.0,1
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,0,1,200,11,3,500.0,0
4,321,87,10,39,42,30,2,396,101,12,48,46,33,0,1,805,40,4,91.5,0
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,1,0,282,421,25,750.0,1


Adesso il dataset è pronto.
Implementando un modello classico, il framework ci avverte che siamo in presenza di forte colinearità.

In [69]:
import statsmodels.api as sm

y = hitters["Salary"]
X = hitters.loc[:,hitters.columns != y.name]

model = sm.OLS(y, X)

Adesso utilizziamo il **LASSO** per selezionare solo le variabili significative.

In [71]:
lasso = model.fit_regularized(
    method='elastic_net',
    alpha=5,
    L1_wt=1.0, #Il peso a 1 è equivalente al LASSO, se lo mettiamo a 0 equivale a RIDGE
)
lasso.params


AtBat           -0.595311
Hits             2.855634
HmRun           -1.579795
Runs             0.245337
RBI              0.107980
Walks            4.559313
Years          -19.148600
CAtBat           0.074372
CHits            0.238804
CHmRun           1.427871
CRuns           -0.030290
CRBI             0.000000
CWalks          -0.516402
League_A       -16.119549
Division_E     117.944676
PutOuts          0.262864
Assists          0.151545
Errors          -5.245978
NewLeague_A      0.000000
dtype: float64